In [18]:
import sys
sys.path.append('../')

import copy
import glob

from typing import Tuple, Union, List
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm as tqdm
from catboost import CatBoostRegressor, Pool, metrics, cv

import torch
import torch.nn as nn

from torch_geometric.utils.convert import from_networkx

from src.utils import *
from dataset import *
from src.scalers import StandardScaler as Scaler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from IPython.display import clear_output

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
G = nx.readwrite.read_gpickle('data/network.gpickle')

In [20]:
node_attrs = list(next(iter(G.nodes(data=True)))[-1].keys())
node_attrs = node_attrs[3:]
# node_attrs = node_attrs[-20:-18] + node_attrs[3:5]
pyg_graph = from_networkx(G, group_node_attrs=node_attrs)
pyg_graph
print(node_attrs)

['labor_force_rate', 'housing_sales_num_Total:', 'housing_sales_num_Total:!!Less than $10,000', 'housing_sales_num_Total:!!$10,000 to $14,999', 'housing_sales_num_Total:!!$15,000 to $19,999', 'housing_sales_num_Total:!!$20,000 to $24,999', 'housing_sales_num_Total:!!$25,000 to $29,999', 'housing_sales_num_Total:!!$30,000 to $34,999', 'housing_sales_num_Total:!!$35,000 to $39,999', 'housing_sales_num_Total:!!$40,000 to $49,999', 'housing_sales_num_Total:!!$50,000 to $59,999', 'housing_sales_num_Total:!!$60,000 to $69,999', 'housing_sales_num_Total:!!$70,000 to $79,999', 'housing_sales_num_Total:!!$80,000 to $89,999', 'housing_sales_num_Total:!!$90,000 to $99,999', 'housing_sales_num_Total:!!$100,000 to $124,999', 'housing_sales_num_Total:!!$125,000 to $149,999', 'housing_sales_num_Total:!!$150,000 to $174,999', 'housing_sales_num_Total:!!$175,000 to $199,999', 'housing_sales_num_Total:!!$200,000 to $249,999', 'housing_sales_num_Total:!!$250,000 to $299,999', 'housing_sales_num_Total:!!$

In [21]:
ds = torch.load('datasets/2021-01-22.dat')

node_to_idx = np.vectorize(dict(zip(pyg_graph.id, range(len(pyg_graph.id)))).get)
X = pyg_graph.x[node_to_idx(ds.X)].cpu().numpy()
y = ds.targets

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=42)

In [22]:
model = CatBoostRegressor(
    loss_function=metrics.RMSE(),
    random_seed=42,
    # depth=3,
    # learning_rate=2e-3,
    eval_metric=metrics.R2()
)

In [23]:
model.fit(X_train, y_train, eval_set=(X_validation, y_validation), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.032703
0:	learn: 0.0076263	test: -0.0077813	best: -0.0077813 (0)	total: 1.09ms	remaining: 1.09s
1:	learn: 0.0197533	test: -0.0087779	best: -0.0077813 (0)	total: 2.21ms	remaining: 1.1s
2:	learn: 0.0267129	test: -0.0059822	best: -0.0059822 (2)	total: 3.31ms	remaining: 1.1s
3:	learn: 0.0360702	test: -0.0009301	best: -0.0009301 (3)	total: 4.5ms	remaining: 1.12s
4:	learn: 0.0400530	test: 0.0006101	best: 0.0006101 (4)	total: 5.51ms	remaining: 1.1s
5:	learn: 0.0491390	test: 0.0037341	best: 0.0037341 (5)	total: 6.63ms	remaining: 1.1s
6:	learn: 0.0605836	test: -0.0010659	best: 0.0037341 (5)	total: 7.64ms	remaining: 1.08s
7:	learn: 0.0658195	test: 0.0001809	best: 0.0037341 (5)	total: 8.48ms	remaining: 1.05s
8:	learn: 0.0782708	test: 0.0038344	best: 0.0038344 (8)	total: 9.36ms	remaining: 1.03s
9:	learn: 0.0836787	test: 0.0045420	best: 0.0045420 (9)	total: 10.4ms	remaining: 1.03s
10:	learn: 0.0868415	test: 0.0047626	best: 0.0047626 (10)	total: 11.3ms	remaining: 1.02s
11:	lea